In [1]:
import os
os.chdir('../..')

In [41]:
from classes.matching import Matching
from classes.helpers import *
import numpy as np
import json
import pandas as pd
from datetime import datetime
import gzip

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

data_folder = '../data/'

notext = {'ba': 'nan', 'rb': ''}


# Create files for valid ratings

In [36]:
for key in ['ba', 'rb']:
    print('Parse {} ratings'.format(key.upper()))
    gen = parse(data_folder + key + '/ratings.txt.gz')
    
    file_ = gzip.open(data_folder + key + '/ratings_valid.txt.gz', 'wb')

    for item in gen:
        text = item['text']
        if text != notext[key]:
            for key_dct in item.keys():
                file_.write('{}: {}\n'.format(key_dct, item[key_dct]).encode('utf-8'))
            file_.write('\n'.encode('utf-8'))
            
    file_.close()

Parse BA ratings
Parse RB ratings


# Create files for valid ratings of matched beers

In [37]:
df = pd.read_csv(data_folder + 'matched/beers.csv', header=[0,1])
beer_ids = {'ba': np.array(df['ba']['beer_id']), 'rb': np.array(df['rb']['beer_id'])}

In [38]:
for key in ['ba', 'rb']:
    print('Parse {} ratings'.format(key.upper()))
    gen = parse(data_folder + key + '/ratings_valid.txt.gz'.format(key))
    
    file_ = gzip.open(data_folder + 'matched/ratings_valid_{}.txt.gz'.format(key), 'wb')

    for item in gen:
        if int(item['beer_id']) in beer_ids[key]:
            for key_dct in item.keys():
                file_.write('{}: {}\n'.format(key_dct, item[key_dct]).encode('utf-8'))
            file_.write('\n'.encode('utf-8'))
            
    file_.close()

Parse BA ratings
Parse RB ratings


# Add the new number of ratings for the matched beers

In [46]:
ratings = {'ba': {}, 'rb': {}}
for key in ['ba', 'rb']:
    print('Parse {} ratings'.format(key.upper()))
    gen = parse(data_folder + 'matched/ratings_valid_{}.txt.gz'.format(key))
    
    for item in gen:
        beer_id = int(item['beer_id'])
        
        if beer_id not in ratings[key].keys():
            ratings[key][beer_id] = []
        ratings[key][beer_id].append(float(item['rating']))

Parse BA ratings
Parse RB ratings


In [47]:
for key in ['ba', 'rb']:
    df = pd.read_csv(data_folder + key + '/beers.csv')
    
    nbrs = []
    avg = []
    for i in df.index:
        beer_id = df.iloc[i]['beer_id']
        if beer_id not in ratings[key].keys():
            nbrs.append(0)
            avg.append(np.nan)
        else:
            nbrs.append(len(ratings[key][beer_id]))
            avg.append(np.mean(ratings[key][beer_id]))
                
    df['nbr_matched_valid_ratings'] = nbrs
    df['avg_matched_valid_ratings'] = avg

    df.to_csv(data_folder + key + '/beers.csv', index=False)

In [48]:
matcher = Matching()
matcher.match_beers()

Matching the beers
